# Imports

In [ ]:
import pandas as pd
import numpy as np
from caits.dataset._dataset3 import DatasetArray, DatasetList
from caits.filtering import filter_butterworth
from caits.fe import mean_value, std_value
from caits.fe import melspectrogram

# Dataset Array test

## Dataset

In [ ]:
data = pd.read_csv("data/AirQuality.csv", sep=";", decimal=",")
data

In [ ]:
data_X = data.iloc[:, 2:-2]
data_X = data_X.fillna(data_X.mean())
data_y = data.iloc[:, -2:]
data_y = data_y.fillna(data_y.mean())

In [ ]:
data_X

In [ ]:
data_y

In [ ]:
datasetArrayObj = DatasetArray(data_X, data_y)

In [ ]:
len(datasetArrayObj)

In [ ]:
datasetArrayObj

In [ ]:
datasetArrayObj[3]

In [ ]:
datasetArrayObj.y

In [ ]:
for i, row in enumerate(datasetArrayObj):
    print(i)

In [ ]:
for i, batch in enumerate(datasetArrayObj.batch(10)):
    print(i)

In [ ]:
train_obj, test_obj = datasetArrayObj.train_test_split()

In [ ]:
len(train_obj), len(test_obj)

In [ ]:
newDatasetArrayObj = train_obj + test_obj
len(newDatasetArrayObj)

In [ ]:
train_obj, test_obj = datasetArrayObj.train_test_split(random_state=42)

In [ ]:
len(train_obj), len(test_obj)

In [ ]:
newDatasetArrayObj = train_obj + test_obj
len(newDatasetArrayObj)

In [ ]:
datasetArrayObj.to_dict()

In [ ]:
datasetArrayObj.apply(filter_butterworth, fs=200, filter_type='lowpass', cutoff_freq=50)

# DatasetList

## Dataset

In [ ]:
from caits.loading import csv_loader

data = csv_loader("data/GestureSet_small")

In [ ]:
X, y, id = data["X"], data["y"], data["id"]

In [ ]:
datasetListObj = DatasetList(X, y, id)
datasetListObj

In [ ]:
len(datasetListObj)

In [ ]:
datasetListObj[3]

In [ ]:
for i, row in enumerate(datasetListObj):
    print(i)

In [ ]:
for i, batch in enumerate(datasetListObj.batch(10)):
    print(i)

In [ ]:
train_obj, test_obj = datasetListObj.train_test_split()

In [ ]:
len(train_obj), len(test_obj)

In [ ]:
newDatasetListObj = train_obj + test_obj
len(newDatasetListObj)

In [ ]:
train_obj, test_obj = datasetListObj.train_test_split(random_state=42)
len(train_obj), len(test_obj)

In [ ]:
newDatasetListObj = train_obj + test_obj
len(newDatasetListObj)

In [ ]:
datasetListObj.apply(filter_butterworth, fs=200, filter_type='lowpass', cutoff_freq=50)

# Pipeline steps test

## DatasetArray

In [ ]:
from caits.transformers._func_transformer_v2 import FunctionTransformer
from caits.transformers._feature_extractor_v2 import FeatureExtractor
from caits.transformers._func_transformer_2d_v2 import FunctionTransformer2D
from caits.transformers._feature_extractor_2d_v2 import FeatureExtractor2D

functionTransformer = FunctionTransformer(filter_butterworth, fs=200, filter_type='lowpass', cutoff_freq=50)
transformedArray = functionTransformer.fit_transform(datasetArrayObj)

In [ ]:
datasetArrayObj.X

In [ ]:
transformedArray.X

In [ ]:
featureExtractor = FeatureExtractor([
    {
        "func": mean_value,
        "params": {}
    },
    {
        "func": std_value,
        "params": {
            "ddof": 0
        }
    }
])

In [ ]:
tmp = featureExtractor.fit_transform(datasetArrayObj)
tmp.keys(), len(tmp["mean_value"]), type(tmp["mean_value"]), tmp["mean_value"].shape

In [ ]:
datasetArrayObj.dict_to_dataset(tmp)

In [ ]:
featureExtractor2D = FunctionTransformer2D(melspectrogram, n_fft=100, hop_length=10)

In [ ]:
tmp = featureExtractor2D.fit_transform(datasetArrayObj)
type(tmp), tmp.shape

# DatasetList

In [ ]:
functionTransformer = FunctionTransformer(filter_butterworth, fs=200, filter_type='highpass', cutoff_freq=50)
transformedList = functionTransformer.fit_transform(datasetListObj)
transformedList

In [ ]:
datasetListObj.X

In [ ]:
transformedList.X

In [ ]:
tmp = featureExtractor.fit_transform(datasetListObj)
tmp.keys(), len(tmp["mean_value"]), type(tmp["mean_value"][0]), tmp["mean_value"][0].shape

In [ ]:
tmpToDataset = datasetListObj.dict_to_dataset(tmp)

In [ ]:
tmpToDataset[0]

In [ ]:
featureExtractor2D = FunctionTransformer2D(melspectrogram, n_fft=10, hop_length=10)

In [ ]:
tmp = featureExtractor2D.fit_transform(datasetListObj)
type(tmp), len(tmp), type(tmp[0]), tmp[0].shape